# 부동소수점 포맷의 제약 사항
float 값의 문제점은 10진수 포맷으로 출력되지만, 내부적으로는 이진수로 저장된다는 점이다.  
예를 들어 10진수 2.5는 10.1로 정확하게 이진수 부동소수점으로 출력할 수 있다.  
하지만 0.3과 같은 10진수 소수점은 10분의 3으로 저장해야 하는데 아무리 많은 소수를 사용해도 정확한 이진수 포맷으로 저장할 수 없다.(2의 제곱이 아니니까)

In [4]:
print(0.1 + 0.1 + 0.1)
print(0.6 + 0.3 + 0.1) ## 정확히 1.0이 되어야 한다.
print((0.6 + 0.3 + 0.1) / 2) ## 정확히 0.5가 되어야 한다.

0.30000000000000004
0.9999999999999999
0.49999999999999994


위 결과들은 수학적으로 틀렸으며 부동소수점 연산이 10분의 1과 같은 분수를 다룰 때마다 아주 작은 반올림 오류가 발생할 수 있다는 사실을 나타낸다.  
물론 이러한 오류를 무시하는 일이 문제가 되지 않는다. 프로그래밍에서는 부동소수점으로 작업할 때 작은 오류들을 허용한다고 가정하기 때문이다.  

# Decimal
위에서 확인한 문제는 round함수를 사용해 반올림을 하면 제거할 수 있는 오류다.  
하지만 round 함수에 계속 의지하지 않고 더 나은 결과를 얻고자 할 때 Decimal 클래스를 사용할 수 있다.

In [11]:
from decimal import Decimal

my_dec = Decimal() ## Decimal 인스턴스의 기본값은 0이다.
print(my_dec)

d = Decimal('0.1') ## Decimal 변수를 초기화할 때 텍스트 문자열을 사용.
print(d + d + d)

d_f = Decimal(0.1) ## float 타입으로 객체를 초기화 한다면??
d_f

0
0.3


Decimal('0.1000000000000000055511151231257827021181583404541015625')

초기화할 때 문자열 "0.1"이 아니라 부동소수점 0.1을 사용하면, 앞서 언급한 대로 부동소수점 값 자체가 이미 반올림 에러를 포함하고 있어서,  
문자열 타입으로 작성한 값으로 초기화하여 "나는 문자열이 표현하고 있는 숫자를 실제 소수점으로 저장하려 한다." 라고 하는 것이 좋다.

In [15]:
## Decimal 클래스는 자릿수를 유지한다.
## 소수점 이하 두 자리의 소수를 갖는 Decimal 인스턴스로 연산을 수행하면 끝이 0으로 끝나더라도 두 자릿수는 유지된다.
d1, d2 = Decimal("0.10"), Decimal("0.20")
print(d1 + d2)

d3, d4 = Decimal("0.020"), Decimal("0.030")
print(d1 * d3) ## 곱하기를하면 소수점 이하 자릿수는 유지되는 것이 아니라 증가한다.
print(round(d1 * d3, 3)) ## round 함수는 소수점 이하 자릿수를 기존 대수대로 조정이 가능하다.

0.30
0.00200
0.002


In [16]:
d = Decimal(5)  ## 정수값을 사용하여 Decimal 객체를 정확하게 초기화할 수 있다.
d += 2 ## Decimal 객체는 정수와 자유롭게 더하거나 곱할 수 있다.
print(d)

7


In [17]:
d = Decimal("7.0")
d += 1.53 ## 부동소수점 값을 Decimal 객체에 더하거나 곱하면 에러가 발생한다.
## 부동소수점 값을 Decimal 객체로 변환하여 연산을 수행할 것.

TypeError: unsupported operand type(s) for +=: 'decimal.Decimal' and 'float'

In [18]:
from decimal import Decimal

class Money:
    def __init__(self, v="0", units="USD"):
        self.dec_amt = Decimal(v)
        self.units = units

In [19]:
m1 = Money("0.10")
print(m1.dec_amt, m1.units)

0.10 USD


In [21]:
from decimal import Decimal

class Money:
    def __init__(self, v="0", units="USD"):
        self.dec_amt = Decimal(v)
        self.units = units

    def __str__(self):
        s = str(self.dec_amt) + " " + self.units
        return s

In [23]:
m1 = Money("5.00", "CAD")
print(m1)
m1

5.00 CAD


In [24]:
from decimal import Decimal

class Money:
    def __init__(self, v="0", units="USD"):
        self.dec_amt = Decimal(v)
        self.units = units

    def __str__(self):
        s = str(self.dec_amt) + " " + self.units
        return s

    def __repr__(self):
        s = ('Money(' + str(self.dec_amt) + " " + self.units + ')')
        return s

In [25]:
m1 = Money("5.00", "CAD")
print(m1)
m1

5.00 CAD


Money(5.00 CAD)

In [41]:
from decimal import Decimal

class Money:
    exch_dict = {
        "USDCAD" : Decimal("0.75"), "USDEUR" : Decimal("1.16"),
        "CADUS" : Decimal("1.33"), "CADEUR" : Decimal("1.54"),
        "EURUSD" : Decimal("0.86"), "EURCAD" : Decimal("0.65")
    }

    def __init__(self, v="0", units="USD"):
        self.dec_amt = Decimal(v)
        self.units = units

    def __str__(self):
        s = str(self.dec_amt) + " " + self.units
        return s

    def __repr__(self):
        s = ('Money(' + str(self.dec_amt) + " " + self.units + ')')
        return s

    def __add__(self, other):
        if self.units != other.units:
            r = Money.exch_dict[self.units + other.units] ## str을 합쳐 dict에 해당하는 키 포맷으로 만들기
            m = Money(self.dec_amt + (other.dec_amt * r), self.units)
        else:
            m = Money(self.dec_amt + other.dec_amt, self.units)

        m.dec_amt = round(m.dec_amt, 2)

        return m

In [42]:
m1 = Money("1.00", "USD")
m2 = Money("99.00", "CAD")

res = m1 + m2
print(res)

75.25 USD


# Decimal 클래스를 상속 받아 Money 클래스 만들기

decimal 클래스는 불변 타입이기 때문에 통화 단위 units에 대한 초기화 설정을 Money클래스에서 초기화해야 한다.  
즉, Money 클래스가 Decimal 클래스에 추가한 별도 속성이다. 하지만 언급한대로 Decimal은 불변 클래스이다.  
따라서 Money 클래스의 \_\_new\_\_ 메서드를 작성해야한다. 이를 통해 Money 클래스에서 상속 받을 Decimal 클래스의 유산은 \_\_new\_\_에 의해 제어된다.

In [ ]:
from decimal import Decimal

class Money(Decimal):
    ## __new__ 함수에 super()를 사용하여 상위 클래스의 __new__ 함수를 호출.
    ## 이 메서드는 상위 클래스 안에서 유래한 클래스를 초기화한다.
    def __new__(cls, v, units="USD"):
        return super(Money, cls).__new__(cls, v) ## 인수는 하위 클래스 이름, cls는 해당 클래스의 참조.
        ## 상위 클래스 버전의 __new__가 반환하는 값을 그대로 반환해야 한다.

    def __init__(self, v, units="USD"):
        self.units = units